In [1]:
import cv2
import numpy as np

In [21]:
cap = cv2.VideoCapture(1)

def play(func=None, vid=cap):
    while True:
        success, frame = vid.read()
        if func:
            frame = func(frame)
        
        cv2.imshow('video', frame)
        if cv2.waitKey(16) & 0XFF==ord('q'):
            break
    cv2.destroyAllWindows()
play()

In [13]:
def preProcess(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.GaussianBlur(img, (3,3), 1)
    img = cv2.Canny(img, 144, 144)
    img = cv2.dilate(img, (3, 3), iterations = 2)
    img = cv2.erode(img, (3, 3), iterations = 1)
    return img
play(preProcess)

In [87]:
def get_box(mask):
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    w, h = 408, 577
    maxApprox = np.array([[[0,0]], [[w,0]], [[0,h]], [[w,h]]])
    maxarea = 0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < 5000: 
            continue
        peri = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.02*peri, True)
        if len(approx) != 4:
            continue
        if area > maxarea:
            maxarea = area
            maxApprox = approx
        
    return maxApprox

In [88]:
def get_warp(img):
    cp = img.copy()
    img = preProcess(img)
    box = get_box(img)
#     box.reshape((4,2))

    nb = []
    for t in box:
        nb.append(t[0])
    box = nb
#     print (box, type(box))

    box = sorted(box, key = lambda t: t[0])
    box = sorted(box, key = lambda t: t[1])
    
    w, h = 408, 577
    matrix = cv2.getPerspectiveTransform(np.float32(box), np.float32([[0, 0], [w, 0], [0, h], [w, h]]))
    return cv2.warpPerspective(cp, matrix, (h, w))
                                         

In [89]:
play(get_warp)